# 处理PHENICX数据集

https://www.upf.edu/web/mtg/phenicx-anechoic

它说应该看txt标注，所以下面的代码是txt到midi

In [ ]:
import os
import subprocess
import sys
sys.path.append('..')
from utils.midiarray import annotation2midi, midi_merge, midi2numpy
import numpy as np

dataset_folder = r"..\data\PHENICX-Anechoic"
output_folder = r".\PHENICX_processed"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

annotations_folder = os.path.join(dataset_folder, "annotations")
audio_folder = os.path.join(dataset_folder, "audio")

pieces = os.listdir(audio_folder)
print(pieces)

instrument_mapping = {
    "bassoon": 70,
    "cello": 42,
    "clarinet": 71,
    "doublebass": 43,
    "flute": 73,
    "horn": 60,
    "oboe": 68,
    "trumpet": 56,
    "viola": 41,
    "violin": 40
}

# 目标音频
fs = 22050
hop = 256
time_step = hop / fs

In [ ]:
def mix_audios(output_path, fs, *input_paths):
    input_files = []
    for input_path in input_paths:
        input_files.extend(['-i', input_path])
    
    filter_complex = f"amix=inputs={len(input_paths)}:duration=longest"
    
    command = ['ffmpeg', *input_files, '-filter_complex', filter_complex, '-ar', str(fs), output_path]
    
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while mixing audios: {e}")
        exit()

In [ ]:
for piece in pieces:
    audio_path = os.path.join(audio_folder, piece)
    annotation_path = os.path.join(annotations_folder, piece)

    output_path = os.path.join(output_folder, piece)
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # 音频合并
    all_audios = [os.path.join(audio_path, wav_name) for wav_name in os.listdir(audio_path)]
    mix_audios(os.path.join(output_path, f"{piece}.wav"), fs, *all_audios)

    # midi制作
    midis = []
    txt_files = [f for f in os.listdir(annotation_path) if f.endswith('.txt') and not f.endswith('_o.txt')]
    for txt_file in txt_files:
        txt_path = os.path.join(annotation_path, txt_file)
        m = annotation2midi(txt_path, ["onset", "offset", "name"], sep = ",", instrument=instrument_mapping[txt_file[:-4]])
        midis.append(m)
    midi_path = os.path.join(output_path, f"{piece}.mid")
    midi_all = midi_merge(midis)
    midi_all.save(midi_path)
    np.save(os.path.join(output_path, f"{piece}.npy"), midi2numpy(midi_all, time_step))
